In [1]:
# Data Collection

!pip install kaggle --upgrade
!pip3 install Pillow

import kaggle
import os
import json
import random
from datetime import datetime
from PIL import Image, ImageStat
from PIL.ExifTags import TAGS
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)

def get_predominant_colors(image_file, numcolors=3, resize=150, output = []):
    # Resize image to speed up processing
    img = Image.open(image_file)
    img = img.copy()
    img.thumbnail((resize, resize))
 
    # Reduce to palette
    paletted = img.convert('P', palette=Image.ADAPTIVE, colors=numcolors)
 
    # Find dominant colors
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=True)
    colors = list()
    for i in range(numcolors):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index*3:palette_index*3+3]
        colors.append(tuple(dominant_color))
    return colors

def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []    
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)    
    distance, index = kdt_db.query(rgb_tuple)
    return int(index)

ModuleNotFoundError: No module named 'webcolors'

In [ ]:
## You have to collect and download a set of images. You have the following tasks to program, automating the process as much as possible:
### 1. Create a folder called images.

!mkdir -p ./images
!mkdir -p ./metadata

### 2. Download open-licensed images to the folder images (minimum 100 images).

kaggle.api.authenticate()
kaggle.api.dataset_download_files('nielspace/pexels-mountain-images', path='./images', unzip=True)    
    
### 3. Save metadata of every image like image size, image format (.jpeg, .png, etc.), image orientation (landscape, portrait, square, etc.), creation date, camera model, etc. in one or more JSON files. You can make use of the Exif information present in the image files.

directory = './images/Mountain'
metadata = {}
orientation = ""
images_data =[]
for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        image_data = {}
        path, file_extension = os.path.splitext(f)
        time_creation = os.path.getmtime(f)
        im = Image.open(f)
        size = im.size
        colors = {}
        predominant_colors = get_predominant_colors(directory + "/" + filename)
        colors["predominant_colors"] = []
        for predominant_color in predominant_colors:
            colors["predominant_colors"].append(convert_rgb_to_names(predominant_color))
        for idx,val in enumerate(["red","green","blue"]):
            colors[val] = Image.Image.getextrema(im)[idx]
            colors[val] = colors[val] + (ImageStat.Stat(im).median[idx],)
        if size[0]>size[1] :
            orientation = "landscape"
        elif size[0] == size[1] :
            orientation = "square"
        else :
            orientation = "portrait"
        metadata[filename] = {
            "file_extension": file_extension,
            "creation_date": datetime.fromtimestamp(time_creation/1000.0).strftime("%m/%d/%Y, %H:%M:%S"),
            "size": {
                "width":size[0],
                "height":size[1]
            },
            "orientation": orientation,
            "colors": {
                "predominant_colors":{
                    "first_color":colors["predominant_colors"][0],
                    "second_color":colors["predominant_colors"][1],
                    "third_color":colors["predominant_colors"][2]
                },
                "red": {
                    "minimum":colors["red"][0],
                    "maximum":colors["red"][1],
                    "median":colors["red"][2]
                },
                "green": {
                    "minimum":colors["green"][0],
                    "maximum":colors["green"][1],
                    "median":colors["green"][2]
                },
                "blue": {
                    "minimum":colors["blue"][0],
                    "maximum":colors["blue"][1],
                    "median":colors["blue"][2]
                }
            }
        }


with open("./metadata/metadata.json", "w") as outfile:
    json.dump(metadata, outfile, indent=4)

In [2]:
# Labeling and Annotation

directory = './images/Mountain'
data_path = "./metadata/metadata.json"


if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)
        
for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        tags = {}
        tags["like"] = random.randint(0,100)
        tags["hashtag"] = "moutain"
        json_data[filename]["tags"] = tags
        
with open("./metadata/metadata.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)

In [4]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Data Analyses

numb_users = 1
directory = './images/Mountain'
data_path = "./metadata/metadata.json"
rand = 0

# json_data represents the metadata for each file
if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)


# Creating a profile for users with liked images
def create_user_profile(json_data, directory, method):
    images = []
    for filename in os.listdir(directory) :
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            if (method == "rand"):
                rand = random.randint(1, 5)
                if rand == 1 :
                    images.append(filename)
            elif (method == "orientation"):
                if (json_data[filename]["orientation"] == "landscape"):
                    images.append(filename)
            elif (method == "color"):
                if (json_data[filename]["colors"][0][0] >= 155):
                    images.append(filename)
            else :
                print("Please enter a valid method (rand | orientation | color)")
    return images
                
profile_user_rand = create_user_profile(json_data, directory, "rand") 
profile_user_orientation = create_user_profile(json_data, directory, "orientation")

In [33]:
# Creating dataframes to predict what the user might like

def get_result_dataframe_per_profile(data, profile_user):
    result = []
    for i in data:
        if i in profile_user:
            result.append('Favorite')
        else:
            result.append('NotFavorite')
    return pd.DataFrame(result, columns=['Liked'])

def get_dataframe_learning(data):
    res = []
    for i in data:        
        res.append([data[i]["colors"][0][0],
                  data[i]["colors"][0][1],
                  data[i]["colors"][0][2],
                  data[i]["orientation"], 
                  data[i]["size"][0],
                  data[i]["size"][1]
                 ])
    return pd.DataFrame(res, columns=['Red', 'Green', 'Blue', 'Orientation', 'Width', 'Height'])
    
def get_model(model, df, rf):
    if (model == "dtc"):
        res = tree.DecisionTreeClassifier()
        res = res.fit(df, rf)
    elif (model == "rfc"):
        res = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=0)
        res = res.fit(df, rf.values.ravel())
    return res

def get_dataframe_prediction(data, label_encoder):
    res = []
    for j in data :
         res.append([data[j]["colors"][0][0],
              data[j]["colors"][0][1],
              data[j]["colors"][0][2],
              label_encoder.transform([data[j]["orientation"]])[0], 
              data[j]["size"][0],
              data[j]["size"][1]
             ])
    return res

def get_prediction(method_object, label_encoder, json_data_prediction):
    prediction = method_object.predict(get_dataframe_prediction(json_data_prediction, label_encoder)) 
    return prediction

## Taking the first two thirds of our dataset to train our algorithm, and the last third to test the model
json_data_learning = dict(list(json_data.items())[:len(json_data)//3])
json_data_predicting = dict(list(json_data.items())[len(json_data)*2//3:])

## Creating dataframe and resultframe for learning
dataframe = get_dataframe_learning(json_data_learning)
resultframe_rand = get_result_dataframe_per_profile(json_data_learning, profile_user_rand)
resultframe_orientation = get_result_dataframe_per_profile(json_data_learning, profile_user_orientation)

## Generating numerical labels for column that need it
le1 = LabelEncoder()
dataframe['Orientation'] = le1.fit_transform(dataframe['Orientation'])
le2 = LabelEncoder()
resultframe_rand['Liked'] = le2.fit_transform(resultframe_rand['Liked'])
le3 = LabelEncoder()
resultframe_orientation['Liked'] = le3.fit_transform(resultframe_orientation['Liked'])

## DTC
### Model
dtc_rand = get_model("dtc", dataframe, resultframe_rand)
dtc_orientation =  get_model("dtc", dataframe, resultframe_orientation)
### Prediction
prediction_rand_dtc = get_prediction(dtc_rand, le1, json_data_predicting)
prediction_orientation_dtc = get_prediction(dtc_orientation, le1, json_data_predicting)

## RTC
### Model
rfc_rand = get_model("rfc", dataframe, resultframe_rand)
rfc_orientation = get_model("rfc", dataframe, resultframe_rand)
### Prediction
prediction_rand_rfc = get_prediction(rfc_rand, le1, json_data_predicting)
prediction_orientation_rfc = get_prediction(rfc_orientation, le1, json_data_predicting)

## Getting the result dataframe for the whole set and compare the last third with the one our model found
result_all_images_rand = get_result_dataframe_per_profile(json_data, profile_user_rand)
result_all_images_orientation = get_result_dataframe_per_profile(json_data, profile_user_orientation)

## Results
res_rand = accuracy_score(le2.transform(result_all_images_rand[len(result_all_images_rand)*2//3:]), prediction_rand_dtc)
res_orientation = accuracy_score(le3.transform(result_all_images_orientation[len(result_all_images_orientation)*2//3:]), prediction_orientation_dtc)

print("Accurary RAND : " + str(res_rand))
print("Feature importances : " + str(dtc_rand.feature_importances_))

print("Accurary ORIENTATION : " + str(res_orientation))
print("Feature importances : " + str(dtc_orientation.feature_importances_))



# find test split pr sklearn ça découpe les dataset automatiquement
# skleanr accuracy score pour ce que j'ai fait au desssus mais en mieux

Accurary RAND : 0.6176470588235294
Feature importances : [0.31646825 0.06790123 0.35210905 0.10185185 0.16166961 0.        ]
Accurary ORIENTATION : 1.0
Feature importances : [0. 0. 0. 1. 0. 0.]


/mnt/c/Users/V3D/Desktop/COURS/MachineLearningProject/env/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/mnt/c/Users/V3D/Desktop/COURS/MachineLearningProject/env/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
